In [1]:
# Standard imports
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"text.usetex": True})
plt.rc('xtick',labelsize=8)
plt.rc('ytick',labelsize=8)
from tqdm import tqdm

In [2]:
class FGC:
    def __init__(self, L, X, n, gamma, lambd, alpha, seed = 666):
        # Save parameter fields.
        self.L = L
        self.X = X
        self.N = X.shape[0]
        self.n = n
        self.d = X.shape[1]
        
        # Initialize optimization variables as random matrices.
        np.random.seed(seed)
        self.X_tilde = np.random.normal(0, 1, (self.n, self.d))
        
        self.clip = 1e-10
        self.C = np.random.normal(0, 1, (self.N, self.n))
        self.C[self.C < self.clip] = self.clip

        # Save regularization variables.
        self.alpha = alpha
        self.lambd = lambd
        self.gamma = gamma
        
        # Optimization parameters.
        self.num_iter = 0
        self.lr = 1e-5
        self.num_c_iter = 100

    def objective(self):
        f = 0
        
        # Bregman divergence.
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        f += -self.gamma * np.linalg.slogdet(self.C.T @ self.L @ self.C + J)[1]
        
        # Dirichlet energy.
        L_tilde = self.C.T @ self.L @ self.C
        f += np.trace(self.X_tilde.T @ L_tilde @ self.X_tilde)

        # Regularization for C.
        f += self.lambd * np.linalg.norm(self.C @ np.ones((self.n, 1)))**2 / 2
        
        # Regularization for X_tilde.
        f += self.alpha * np.linalg.norm(self.X - self.C @ self.X_tilde)**2 / 2
        
        return f

    def update_X_tilde(self):
        L_tilde = self.C.T @ self.L @ self.C
        A = 2 * L_tilde / self.alpha + self.C.T @ self.C
        self.X_tilde = np.linalg.pinv(A) @ self.C.T @ X

        for i in range(len(self.X_tilde)):
            self.X_tilde[i] = self.X_tilde[i] / np.linalg.norm(self.X_tilde[i])
        
        return None

    def gradient_C(self):
        grad = np.zeros(self.C.shape)
        
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        v = np.linalg.pinv(self.C.T @ self.L @ self.C + J)
        grad += -2*self.gamma * self.L @ self.C @ v
        
        grad += self.alpha * (self.C @ self.X_tilde - self.X) @ self.X_tilde.T
        grad += 2*self.L @ self.C @ self.X_tilde @ self.X_tilde.T
        grad += self.lambd * np.abs(self.C) @ (np.ones((self.n, self.n)))

        return grad

    def update_C(self):
        self.C = self.C - self.lr * self.gradient_C()
        self.C[self.C < self.clip] = self.clip

        for i in range(len(self.C)):
            self.C[i] = self.C[i] / np.linalg.norm(self.C[i],1)
        
        return None

    def fit(self, num_iters):
        loss = np.zeros(num_iters)
        for i in tqdm(range(num_iters)):
            for _ in range(self.num_c_iter):
                self.update_C()
            self.update_X_tilde()
            loss[i] = self.objective()
            self.num_iter += 1
        return (self.C, self.X_tilde, loss)

In [3]:
class GC:
    def __init__(self, L, n, gamma, lambd, alpha, seed = 666, quiet = False):
        # Save parameter fields.
        self.L = L
        self.N = L.shape[0]
        self.n = n
        
        # Initialize optimization variables as random matrices.
        self.clip = 1e-10
        self.C = np.random.normal(0, 1, (self.N, self.n))
        self.C[self.C < self.clip] = self.clip

        # Save regularization variables.
        self.alpha = alpha
        self.lambd = lambd
        self.gamma = gamma
        
        # Optimization parameters.
        self.num_iter = 0
        self.lr = 1e-5
        self.num_c_iter = 100
        
        # Turn off tqdm output.
        self.quiet = quiet

    def objective(self):
        f = 0
        
        # Bregman divergence.
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        f += -self.gamma * np.linalg.slogdet(self.C.T @ self.L @ self.C + J)[1]

        # Regularization for C.
        f += self.lambd * np.linalg.norm(self.C @ np.ones((self.n, 1)))**2 / 2
        
        return f

    def gradient_C(self):
        grad = np.zeros(self.C.shape)
        
        J = np.outer(np.ones(self.n), np.ones(self.n)) / self.n
        v = np.linalg.pinv(self.C.T @ self.L @ self.C + J)
        grad += -2*self.gamma * self.L @ self.C @ v
        grad += self.lambd * np.abs(self.C) @ (np.ones((self.n, self.n)))

        return grad

    def update_C(self):
        self.C = self.C - self.lr * self.gradient_C()
        self.C[self.C < self.clip] = self.clip

        for i in range(len(self.C)):
            self.C[i] = self.C[i] / np.linalg.norm(self.C[i],1)
        
        return None

    def fit(self, num_iters):
        loss = np.zeros(num_iters)
        for i in tqdm(range(num_iters), disable = self.quiet):
            for _ in range(self.num_c_iter):
                self.update_C()
            loss[i] = self.objective()
            self.num_iter += 1
        return (self.C, loss)

In [4]:
def make_LX(G, d, seed = None):
    # Create edge weights
    np.random.seed(seed)
    N = len(G.nodes)
    W = np.zeros((N, N))
    for (x, y) in G.edges:
        W[x][y] = np.random.randint(1,10)
    W = W + W.T

    # Compute Laplacian
    D = np.diag(W @ np.ones((W.shape[0])))
    L = D - W

    # Create features for the synthetic graph
    X = np.random.multivariate_normal(np.zeros(N), np.linalg.pinv(L), d).T
    
    return L, X

def make_erdos_renyi_graph(N, p, d, seed = None):
    G = nx.erdos_renyi_graph(N, p, directed = False)
    return make_LX(G, d, seed)

def make_barabasi_albert_graph(N, m, d, seed = None):
    G = nx.barabasi_albert_graph(n = N, m = m, seed = seed)
    return make_LX(G, d, seed)

def make_watts_strogatz_graph(N, k, p, d, seed = None):
    G = nx.watts_strogatz_graph(N, k, p, seed = seed)
    return make_LX(G, d, seed)

def make_random_geometric_graph(N, r, d, seed = None):
    # Make graph
    G = nx.random_geometric_graph(N, r, seed = seed)
    return make_LX(G, d, seed)

In [5]:
def reconstruction_error(L, L_hat):
    return np.linalg.norm(L - L_hat)

In [37]:
L, X = make_erdos_renyi_graph(100, 0.9, 500)
#L, X = make_barabasi_albert_graph(N, 20, 500)
#L, X = make_watts_strogatz_graph(N, 20, 0.1, 500)
#L, X = make_barabasi_albert_graph(N, 20, 500)
#L, X = make_random_geometric_graph(100, 0.1, 500)

In [38]:
n = 2
num_iter = 100

fgc = GC(L, n, 500, 500, X.shape[1]/2) 
C, loss = fgc.fit(num_iter)

100%|██████████| 100/100 [00:08<00:00, 12.07it/s]


In [39]:
loss

array([20193.66178193, 20153.44689702, 20136.499304  , 20129.5046158 ,
       20102.05673057, 20043.78325151, 20020.62458534, 20006.77337932,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
       20002.42262967, 20002.42262967, 20002.42262967, 20002.42262967,
      

In [40]:
P = np.linalg.pinv(C)
H = np.round(C @ P, 2)

In [41]:
H

array([[ 0.02,  0.  ,  0.02, ...,  0.02,  0.02, -0.  ],
       [ 0.  ,  0.02,  0.  , ...,  0.  ,  0.  ,  0.02],
       [ 0.02,  0.  ,  0.02, ...,  0.02,  0.02,  0.  ],
       ...,
       [ 0.02,  0.  ,  0.02, ...,  0.02,  0.02,  0.  ],
       [ 0.02,  0.  ,  0.02, ...,  0.02,  0.02, -0.  ],
       [-0.  ,  0.02,  0.  , ...,  0.  , -0.  ,  0.02]])

In [47]:
H - np.round(np.linalg.pinv(H), 2)

array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  , -0.  ],
       [ 0.  , -0.01,  0.  , ...,  0.  ,  0.  , -0.01],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  , -0.  ],
       [-0.  , -0.01,  0.  , ...,  0.  ,  0.  , -0.01]])